# Solutions to linear wave advection problem

This is an implementation of the linear wave advection problem pre-class assignment.

In [ ]:
# User sets stuff in this cell

N = 100               # grid size
u = -0.1               # wave speed, can be positive or negative
C = 1.0               # CFL, should be less than 1 for completeness
N_periods = 1.0      # could be anything; 0.1 or 1.0 for this assignment
IC_type = 'tophat'    # could be 'Gaussian' or 'tophat'
method  = 'upwind'    # could be 'upwind' or 'FTCS'
save_figure = False    # Save auto-named plot to disk?  True or False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# make a title and file name 

mytitle = "C = " + str(C) + ", Np = " + str(N_periods) + ", IC = " + IC_type + ", method = " + method
myfilename = "C_" + str(C) + "_" + "Np_" + str(N_periods) + "_" + IC_type + "_" + method + ".png"

print("TITLE:      ", mytitle)
print("FILE NAME:  ", myfilename)

# Grid dx
dx = 1.0/N

# final time (take abs. magnitude of velocity or else it'll be negative)
T_final = N_periods/np.abs(u)

# time step (need abs. magnitude of velocity or else it'll be negative)
dt = C*dx/np.abs(u)

In [ ]:
def ICs(N=100,IC_type='tophat'):
    '''
    Initial condition generator.  Inputs are the desired number of grid 
    points (N) and initial conditions type (IC_type, options are 'tophat' 
    and 'Gaussian').
    
    Outputs are an array of positions (mostly useful for plotting) and
    the a values that are advected.
    '''
    
    # 'a' array starts out zero everywhere.
    a = np.zeros(N)
    
    # set up positions, including a step to make sure the points are at
    # the middle of the cell.
    x = np.linspace(0.0,1.0,num=N,endpoint=False)+0.5/N
    
    if IC_type=='tophat':  # top hat ICs
        
        # make a filter array that's True for the region we want 
        # to be 1, then set the array.
        filter_array = np.logical_and( x>0.35 , x<0.65)
        a[filter_array]=1.0

    elif IC_type=='Gaussian':  # Gaussian ICs
        sigma = 0.1
        a = np.exp(-(x-0.5)**2/(2*sigma*sigma))
    else:
        print("You didn't put in a correct IC type option:", IC_type)
    
    return x, a

In [ ]:
def evolve(a,C,u,dx,method='upwind'):
    '''Evolves the linear wave forward one timestep.  Inputs are:
    
    a = array that will be evolved
    C = CFL condition
    u = velocity (can be positive or negative)
    dx = grid spacing
    method = numerical method ('upwind' or 'FTCS')
    
    returns new 'a' array that has been evolved one timestep.
    '''
    
    # make a new array, which we'll modify momentarily.
    a_new = np.zeros_like(a)
    
    if method=='upwind':  # upwind method
        if u>0: # for positive velocity (traveling right), need i-1 and i cells
            a_new[1:] = a[1:]-C*(a[1:]-a[:-1]) 
            a_new[0] = a[0]-C*(a[0]-a[-1])  # take care of BC
        else:   # for negative velocity (traveling left), need i and i+1 cells
            a_new[:-1] = a[:-1]-C*(a[:-1]-a[1:])
            a_new[-1] = a[-1]-C*(a[-1]-a[0]) # take care of BC
        
    elif method=='FTCS':  # forward-time, center-space is symmetric. np.sign() controls direction.
        a_new[1:-1]=a[1:-1]-0.5*C*np.sign(u)*(a[2:]-a[0:-2])
        a_new[0]=a[0]-0.5*C*np.sign(u)*(a[1]-a[-1])   # do left hand side BCs
        a_new[-1]=a[-1]-0.5*C*np.sign(u)*(a[0]-a[-2])  # do right hand side BCs
    else:
        print("This isn't an approved method: ", method)
    
    # return the updated a array
    return a_new

In [ ]:
def makeplot(x,a_orig,a_now,this_title='Output Information',filename='output.png',savefig=True):
    '''
    Makes a plot and (optionally) saves it to a file.  Inputs are:
    x = position array
    a_orig = original set of a values (initial conditions)
    a_now = current time set of a values
    this_title = your title
    filename = desired filename
    savefig = Boolean for whether or not you want your figure saved to a file.
    
    Output is a plot displayed to screen and (optionally) saved to file.
    '''
    plt.plot(x,a_orig,'k-')
    plt.plot(x,a_now,'r--')
    plt.xlabel('x')
    plt.ylabel('a(x)')
    plt.title(this_title)
    if savefig==True:
        plt.savefig(filename,dpi=400)


In [ ]:
# set up initial conditions arrays
x, a = ICs(N,IC_type)

# make a copy of our initial conditions for plotting purposes
a_original = np.copy(a) 

# make a quick plot, just in case
plt.plot(x,a_original,'b-')
plt.xlabel('x')
plt.ylabel('a(x)')
plt.title('Initial conditions')

In [ ]:
# This cell evolves our simulation until the end time, using a timestep dt

# initial time is always zero
t=0.0

# loop until we're done.
# the -dt in the while() is because we add 
# the dt after we do the evolution.
# otherwise, things don't line up correctly.
while(t < T_final-dt):
    
    a = evolve(a,C,u,dx,method)   
    
    t+=dt
    

In [ ]:
# make our plot and (optionally) save it.
makeplot(x,a_original,a,mytitle,myfilename,savefig=save_figure)
